## Filtering SAM 

This job to filter SAM files output from Nextgenmap. It runs `samtools` to convert to BAM and store only human unmapped content.

In [1]:
import pysam
import os
from hops import hdfs
import utils
import sys
from pyspark import SparkContext
import subprocess


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
553,application_1615491195715_0581,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [20]:

args=utils.load_arguments(sys.argv)
#args=utils.load_arguments([0,'hdfs:///Projects/TCGA_viruses/Jupyter/pipeline/settings_DJ.yml'])
if args is not None:
    args=args['Idxstat']
else :
    sys.exit(utils.NO_CONFIG_ERR)
    
sc = SparkContext.getOrCreate()

Started copying hdfs:///Projects/TCGA_viruses/Jupyter/pipeline/settings_DJ.yml to local disk on path /srv/hops/hopsdata/tmp/nm-local-dir/usercache/rFfxgJCqYiFgJW_6YMkFOwNpaYJveIXICcW8hZxwcxQ/appcache/application_1615491195715_0581/container_e41_1615491195715_0581_01_000001/

Finished copying

In [21]:

inputRoot=args['INPUT_BAM']
inputBai=args['INPUT_BAI']
OUTPUT_INDEX=args['OUTPUT_INDEX']
OUTPUT_STAT=args['OUTPUT_STAT']

threads=str(args['THREADS'])

8988

In [12]:

def idx(file):
    filename=os.path.basename(file)
    idx_file=filename.split('.')[0]+'.txt'
    bai_file=filename.split('.')[0]+'.bai'
    
    if not hdfs.exists(os.path.join(OUTPUT_STAT,idx_file)):    
    
        hdfs.copy_to_local(file, overwrite=True)
        hdfs.copy_to_local(os.path.join(inputBai,bai_file))
        ### running indexing if absent ###

        #file=os.path.split(file)[1]

        #print("INFO: Run indexing : ", file)    
        #cmd='samtools index '+file + ' -@ '+ threads
        #subprocess.run(cmd.split(),stdout=subprocess.PIPE)
        

        #pysam.index(file,'-@',threads, catch_stdout=False)

        ### running indexing if absent ###
        

        if os.path.exists(bai_file):
            #hdfs.copy_to_hdfs(bai_file,OUTPUT_INDEX,overwrite=True)
                   

            stdout=pysam.idxstats(filename,'-@',threads, catch_stdout=True)
            with open(idx_file,'w') as f: 
                 f.write(stdout)

            hdfs.copy_to_hdfs(idx_file,OUTPUT_STAT,overwrite=True)
            os.remove(idx_file)
            os.remove(bai_file)


        os.remove(filename)
    else :
        print('skipping file', file)

    return None




    


In [26]:

inputFiles=utils.load_file_names(inputRoot)

In [27]:
len(inputFiles)

775

'TCGA_Data/chunk11'

In [28]:
"""
removed files due to samtools failure for corrupt files
"""
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk4/18c4de42-b42e-44d6-90fc-f4130a2f76d6_gdc_realn_rehead.bam')
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk4/d9e0312b-86fd-4490-9626-4ba2e213d8ac_gdc_realn_rehead.bam')
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk5/d1ee552b-5219-47bd-be51-78d5467c4dd9_gdc_realn_rehead.bam')
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk7/e3a8f442-5157-4153-8ca1-9dfb11b8992c_gdc_realn_rehead.bam')
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk8/536b58ff-f385-4f1d-bb7a-47a52336e09d_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/2ef729a3-ef44-4e8f-ad03-539e86a30862_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/b303f535-1023-4e11-8cf9-fa9b9e81ef3a_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/108dc6d1-5612-4276-a682-0ba4d324cb00_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/a4739807-50c0-4da1-926a-4fc130d734fb_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/ba928d91-8f40-4e77-b67f-f7b5ba5b5520_gdc_realn_rehead.bam')
# inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk9/bcdf0550-9e6f-4500-9473-706df7beeae4_gdc_realn_rehead.bam')


##last chunk
#inputFiles.remove('hdfs://rpc.namenode.service.consul:8020/Projects/TCGA_viruses/TCGA_Data/chunk11/b303f535-1023-4e11-8cf9-fa9b9e81ef3a_gdc_realn_rehead.bam')

""" end remove"""





In [14]:
#inputFiles=['hdfs:///Projects/TCGA_viruses/TCGA_Data/chunk11/5b49dd79-a663-45bb-b56b-8b4b00b68191_gdc_realn_rehead.bam']

In [15]:
# inputFiles=[inputFiles[0] ]
# inputFiles

In [16]:



### convert to bam and filter unmapped sequences
unMapped=sc.parallelize(inputFiles).map(idx).collect()


774